In [1]:
import numpy as np

from data_loaders import *

# Instructions on how to load data

In [10]:
# How to read one file: careful, this might take a long time
# data is by defult QUANTILE scaled so between [0, 1]
a = read_scale(SAMPLE_PATH, no_obs=400000, slice_size=10)
a.shape

(399990, 200)

In [13]:
# How to read best bid and ask prices quickly
buy, sell = read_bid_ask(SAMPLE_PATH, no_obs=400000)
buy

0         3171.8
1         3172.2
2         3169.8
3         3173.2
4         3172.8
           ...  
399995    2985.0
399996    2985.0
399997    2985.0
399998    2983.8
399999    2983.8
Name: B1, Length: 400000, dtype: float64

In [3]:
# How to iterate over all datafiles available
#  the generator loops over all csv files in data/order_books
for a in generate_data(read_scale, no_obs=2000, slice_size=10):
    print(a)
    # Do anything you want with a, like train the model

[[0.85835836 0.85835836 0.86036036 ... 0.21471471 0.32832833 0.33533534]
 [0.88338338 0.88638639 0.88788789 ... 0.         0.         0.91841842]
 [0.91741742 0.90690691 0.90840841 ... 0.         0.         0.        ]
 ...
 [0.50650651 0.50800801 0.50800801 ... 0.75425425 0.9964965  0.        ]
 [0.50650651 0.50800801 0.50800801 ... 0.79129129 0.75975976 0.996997  ]
 [0.50650651 0.50800801 0.50800801 ... 0.79129129 0.78078078 0.996997  ]]
[[0.84684685 0.91321403 0.98307532 ... 0.         0.         0.8973974 ]
 [0.98748749 0.99899544 0.99099099 ... 0.         0.         0.8973974 ]
 [0.98748749 0.99899544 0.99099099 ... 0.         0.         0.        ]
 ...
 [0.5955956  0.48248248 0.58408408 ... 0.         0.         0.        ]
 [0.5955956  0.48248248 0.58408408 ... 0.         0.         0.        ]
 [0.5955956  0.48248248 0.58408408 ... 0.82432432 0.         0.        ]]


# Sample model

In [158]:
from sklearn.neural_network import MLPRegressor
nn = MLPRegressor()
nn.fit(X, X[50:, 0]).score(X[:-50], X[50:, 0])

/Users/fabiopruneri/Documents/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.7576369322474805